# Dataosuces declaration

## IMDB Datasource

In [1]:
from imdb import IMDb
imdb = IMDb()
# [ (x.data['title'], x.data['rating']) for x in imdb.get_top250_movies() ][0:25]

## MovieLens Datasource

In [2]:
datasets = {
    'genome-scores': './data/ml-latest-small/genome-scores.csv',
    'genome-tags': './data/ml-latest-small/genome-tags.csv',
    'links': './data/ml-latest-small/links.csv',
    'movie': './data/ml-latest-small/movies.csv',
    'ratings': './data/ml-latest-small/ratings.csv',
    'tags': './data/ml-latest-small/tags.csv',
}

# Data Profiling

In [3]:
import pandas_profiling
import pandas as pd

def read(dataset):
    print('Reading: ', datasets[dataset])
    return pd.read_csv(datasets[dataset])

In [4]:
from IPython.display import display

# for dataset_k in datasets:
#     df = read(dataset_k)
#     display(df.head())

# Cube creation

In [5]:
links = read('links')
ratings = read('ratings')

Reading:  ./data/ml-latest-small/links.csv
Reading:  ./data/ml-latest-small/ratings.csv


## Facts

In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Dimensions

### Time and Date

In [7]:
from datetime import datetime as dt

mesToTexto = [
    'enero',
    'febrero',
    'marzo',
    'abril',
    'mayo',
    'junio',
    'julio',
    'agosto',
    'septiembre',
    'octubre',
    'noviembre',
    'diciembre',
]

def getDate(ts):
    time = dt.fromtimestamp(ts)
    return {
        'anyo':  time.year,
        'mes':  time.month,
        'dia':  time.day,
        'mes_texto': mesToTexto[time.month-1]
    }

def getHour(ts):
    time = dt.fromtimestamp(ts)
    return {
        'hour':  time.hour,
        'minute':  time.minute,
        'second':  time.second,
    }

(getDate(1112486027), getHour(1112486027))

({'anyo': 2005, 'mes': 4, 'dia': 3, 'mes_texto': 'abril'},
 {'hour': 1, 'minute': 53, 'second': 47})

### Film

In [8]:
def getFilm(movie_id):
    imdbId = links.loc[links['movieId'] == movie_id]['imdbId']
    movie = imdb.get_movie(imdbId)
    return {
        'title': movie.data['title'],
        'year': movie.data['year'],
        'genre': movie.data['genres'][0] if movie.data['genres'] else 'none',
        '_movieId': int(movie_id),
    }

getFilm(29)

{'title': 'The City of Lost Children',
 'year': 1995,
 'genre': 'Fantasy',
 '_movieId': 29}

# ETL

In [9]:
datasets = {
    'genome-scores': './data/ml-20m/genome-scores.csv',
    'genome-tags': './data/ml-20m/genome-tags.csv',
    'links': './data/ml-20m/links.csv',
    'movie': './data/ml-20m/movies.csv',
    'ratings': './data/ml-20m/ratings.csv',
    'tags': './data/ml-20m/tags.csv',
}

In [10]:
import sqlite3

db = sqlite3.connect('./data.db')
cursor = db.cursor()

In [11]:
with open('./modeloA.sql') as schema:
    query = schema.read()
    cursor.executescript(query)
    db.commit()

In [12]:
import pandas_profiling
import pandas as pd

def read(dataset):
    print('Reading: ', datasets[dataset])
    return pd.read_csv(datasets[dataset])

In [13]:
def extractRating(*args):
    yield {
        'movieId': args[1],
        'rating': args[2],
        'timestamp': args[3],
    }

In [14]:
def addDate(rating):
    date = getDate(int(rating['timestamp']))
    fecha = (date['dia'], date['mes_texto'], date['anyo'])
    cursor.execute('SELECT * from Fecha WHERE dia=? AND mes=? AND anyo=?', fecha)
    data = cursor.fetchone()
    if (data==None):
        fecha = ("{}-{}-{}".format(date['anyo'], date['mes'], date['dia']),*fecha)
        cursor.execute('INSERT INTO Fecha (fecha, dia, mes, anyo) VALUES (?, ?, ?, ?)', fecha)
        db.commit()
        rating['dateId'] = cursor.lastrowid
    else:
        rating['dateId'] = data[0]   
    yield rating

In [15]:
def addHour(rating):
    hour = getHour(int(rating['timestamp']))
    hora = (hour['hour'],hour['minute'],hour['second'])
    cursor.execute('SELECT * from Hora WHERE hora=? AND minuto=? AND segundo=?', hora)
    data = cursor.fetchone()
    if (data==None):
        hora = (*hora, rating['timestamp'])
        cursor.execute('INSERT INTO Hora (hora, minuto,segundo, timestamp) VALUES (?, ?, ?, ?)', hora)
        db.commit()
        rating['hourId'] = cursor.lastrowid
    else:
        rating['hourId'] = data[0]   
    yield rating

In [16]:
links = read('links')
def addFilm(rating):
    pelicula = (rating['movieId'],)
    cursor.execute('SELECT * from Pelicula WHERE idNatural=?', pelicula)
    data = cursor.fetchone()
    if (data==None):
        film = getFilm(int(rating['movieId']))
        pelicula = (film['_movieId'], film['title'], film['year'], film['genre'])
        cursor.execute('INSERT INTO Pelicula (idNatural, titulo, anyoLanzamiento, genero) VALUES (?, ?, ?, ?)', pelicula)
        db.commit()
        rating['filmId'] = cursor.lastrowid
    else:
        rating['filmId'] = data[0]   
    yield rating

Reading:  ./data/ml-20m/links.csv


In [17]:
def load(rating):
    voto = (rating['userId'], rating['filmId'],rating['hourId'],rating['dateId'],rating['rating'])
    cursor.execute('INSERT INTO Voto (clvUsuario, clvPelicula, clvHora, clvFecha, nota) VALUES (?, ?, ?, ?, ?)', voto)
    db.commit()
    yield

In [18]:
def extract():
    with open(datasets['ratings']) as f:
        next(f)
        for line in f:        # create a list of lists
            line = line.split(',')
            rating = {}
            rating['userId'] = line[0]
            rating['movieId'] = line[1]
            rating['rating'] = line[2]
            rating['timestamp'] = line[3].strip('\n')
            yield rating

In [ ]:
for a in extract():
    try:
        for a in addDate(a):
            print('DATE',a)
            for a in addHour(a):
                print('HOUR',a)
                for a in addFilm(a):
                    print('FILM',a)
                    for a in load(a):
                        pass
    except:
        print('Error')
        pass